In [ ]:
import os
import glob
import dicom
import numpy as np
import pandas as pd

import tensorflow
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D, ZeroPadding3D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
def get_3d_data(path):
    """Get the 3D data for sample patients"""
    # For each .dcm file
    slices = [dicom.read_file(s) for s in glob.glob(path + '/*.dcm')]
    slices.sort(key=lambda x: int(x.InstanceNumber))
    return np.stack([s.pixel_array for s in slices])

ct = []
for path in glob.glob(os.path.join('../sample_images/', '*')):
    # For each patient
    ct.append(get_3d_data(path))

In [ ]:
# Crop to 100 slices for now
ct = [c[0:100,:,:] for c in ct]

In [ ]:
def get_model():
    model = Sequential()
    model.add(Convolution3D(16, 1,3,3, input_shape=(1,100,512,512), activation='relu'))
    model.add(Convolution3D(32, 1,3,3, activation='relu'))
    model.add(MaxPooling3D(pool_size=(1,2,2)))
    model.add(Convolution3D(32, 1,3,3, activation='relu'))
    model.add(Convolution3D(64, 1,3,3, activation='relu'))
    model.add(MaxPooling3D(pool_size=(1,2,2)))
    model.add(Convolution3D(64, 1,3,3, activation='relu'))
    model.add(Convolution3D(128, 1,3,3, activation='relu'))
    model.add(MaxPooling3D(pool_size=(1,2,2)))
    model.add(Convolution3D(128, 1,3,3, activation='relu'))
    model.add(MaxPooling3D(pool_size=(1,2,2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='softmax'))
    
    model.compile(loss='mean_squared_error', optimizer='adadelta', metrics=['accuracy'])
    return model

In [ ]:
# Split data and labels
labels = pd.read_csv('../stage1_labels.csv')
labels = np.array(labels)

x_train = np.array(ct[0:18])
y_train = labels[0:18,1].reshape((-1, 1))
x_test = ct[19]
y_test = labels[19,1]

In [ ]:
final_images = np.ndarray([18,1,100,512,512], dtype=np.float32)
for i in range(18):
    final_images[i,0] = x_train[i]
final_images.shape

In [ ]:
model = get_model()
model.fit(final_images, y_train, batch_size=24, nb_epoch=1, #10
          show_accuracy=True, verbose=1,
          validation_data=(x_test, y_test))

In [ ]:
print(model.summary())
print(model.weights)